In [1]:
import pandas as pd
import numpy as np

input_dir = 'F:\\八斗学院\\视频\\14期正式课\\00-data//'
out_dir=input_dir+'out//'
'''
读取数据
priors表示用户的历史购买数据
order_products__train表示用户倒数第二天的购买数据
召回中命中的为1，这个用户所有的购买过的记录作为召回商品，
train的数据为最近一天的商品，也就是从这个用户之前购买过所有商品中，
最近一天购买了属于命中了，这样模型倾向于抓住最近用户的购买需求，淡化时间久远的购买兴趣
'''
priors = pd.read_csv(filepath_or_buffer=input_dir + 'order_products__prior.csv', dtype={
    'order_id': np.str,
    'product_id': np.str,
    'add_to_cart_order': np.int,
    'reordered': np.int
})

train = pd.read_csv(filepath_or_buffer=input_dir + 'order_products__train.csv',
                    dtype={
                        'order_id': np.str,
                        'product_id': np.str,
                        'add_to_cart_order': np.int,
                        'reordered': np.int
                    })
orders = pd.read_csv(filepath_or_buffer=input_dir + 'orders.csv',
                     dtype={
                         'order_id': np.str,
                         'user_id': np.str,
                         'eval_set': 'object',
                         'order_number': np.int16,
                         'order_dow': np.int8,
                         'order_hour_of_day': np.int,
                         'days_since_prior_order': np.float32
                     })

products = pd.read_csv(input_dir + 'products.csv', dtype={
    'product_id': np.str,
    'order_id': np.str,
    'aisle_id': np.str,
    'department_id': np.str},
                       usecols=['product_id', 'aisle_id', 'department_id'])

print('prior {}:{}'.format(priors.shape,','.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('train {}: {}'.format(train.shape, ', '.join(train.columns)))

'''
特征处理
'''
# 1 product feat
prod_feat_df=pd.DataFrame()
# 产品销量
prod_feat_df['orders']=priors.groupby(priors.product_id).size().astype(np.int)
# 产品再次被购买量
prod_feat_df['reorders']=priors.groupby('product_id')['reordered'].sum()
# 产品再次购买比例
prod_feat_df['reorder_rate']=(prod_feat_df['reorders']/prod_feat_df['orders']).astype(np.float32)
# 合并product的特征
products=products.join(prod_feat_df,how='inner',on='product_id')
# 设置product_id为index列，drop表示是否删除product_id列 inplace表示是否在原数据上修改
products.set_index('product_id',drop=False,inplace=True)
del prod_feat_df

# 2 历史商品数据关联订单数据
priors=pd.merge(priors,orders,how='inner',on='order_id')

# 3 计算用户特征
# 用户订单特征
usr=pd.DataFrame()
# 每个用户平均订单时间间隔
usr['average_days_between_orders']=orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
# 用户订单数量
usr['nb_orders']=orders.groupby('user_id').size().astype(np.int)

#用户商品特征
users=pd.DataFrame()
# 用户购买商品数量
users['total_items']=priors.groupby('user_id').size().astype(np.int)
# 用户购买商品去重（set）集合
users['all_products']=priors.groupby('user_id')['product_id'].apply(set)
# 用户去重后的商品数量
users['total_distinct_items']=users['all_products'].map(len).astype(np.int)
# users['total_distinct_items']=users['all_products'].apply(len)
users=users.join(usr,on='user_id')
# 用户平均一个订单的商品数量
users['average_basket']=(users['total_items']/users['nb_orders']).astype(np.float)
print('user feat',users.shape)
# 存储用户特征
users.to_csv(path_or_buf=out_dir+'users.csv')

'''4用户和商品的交叉特征'''
print('compute userXproduct f - this is long...')
# user_id+product_id的组合key
priors['user_product'] =priors['user_id']+'_'+priors['product_id']

# 存储商品和用户特征
userXproduct.sum_pos = userXproduct.sum_pos.astype(np.int)
print('user X product feat', len(userXproduct))
del priors.to_csv(path_or_buf=out_dir+'priors.csv')

prior (32434489, 4):order_id,product_id,add_to_cart_order,reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered
user feat (206209, 6)
compute userXproduct f - this is long...


In [2]:
# 定义字典表 key=user_product val(1,2,3):
# 1表示用户购买的该商品数
# 2表示最近一个订单
# 3表示购物车位置累加
d=dict()
for idx,row in priors.iterrows():
    user_product=row.user_product
    if user_product not in d:
        d[user_product]=(
            1,
            (row['order_number'],row['order_id']),
            row['add_to_cart_order']
        )
    else:
        d[user_product]=(
            d[user_product][0]+1,
            max(d[user_product][1],(row['order_number'],row['order_id'])),
            row['add_to_cart_order']+d[user_product][2]
        )

In [3]:
d


{'202279_33120': (5, (8, '104690'), 10),
 '202279_28985': (5, (6, '132412'), 16),
 '202279_9327': (1, (3, '2'), 3),
 '202279_45918': (5, (7, '2382766'), 24),
 '202279_30035': (3, (7, '2382766'), 14),
 '202279_17794': (7, (7, '2382766'), 25),
 '202279_40141': (5, (6, '132412'), 29),
 '202279_1819': (2, (3, '2'), 19),
 '202279_43668': (3, (6, '132412'), 20),
 '205970_33754': (17, (25, '368699'), 86),
 '205970_24838': (14, (25, '368699'), 58),
 '205970_17704': (13, (25, '368699'), 72),
 '205970_21903': (14, (25, '368699'), 55),
 '205970_17668': (6, (16, '3'), 54),
 '205970_46667': (13, (25, '368699'), 69),
 '205970_17461': (4, (24, '2214773'), 35),
 '205970_32665': (6, (24, '2214773'), 45),
 '178520_46842': (17, (55, '1609396'), 99),
 '178520_26434': (4, (42, '2715951'), 19),
 '178520_39758': (19, (56, '1253720'), 169),
 '178520_27761': (49, (56, '1253720'), 401),
 '178520_10054': (29, (56, '1253720'), 235),
 '178520_21351': (33, (55, '1609396'), 301),
 '178520_22598': (12, (55, '1609396'

In [4]:
userXproduct=pd.DataFrame.from_dict(d,orient='index')


In [7]:
userXproduct.head()

,nb_products,last_order_id,sum_pos
202279_33120,5,"(8, 104690)",10
202279_28985,5,"(6, 132412)",16
202279_9327,1,"(3, 2)",3
202279_45918,5,"(7, 2382766)",24
202279_30035,3,"(7, 2382766)",14


In [6]:
del d
# 设置列名
userXproduct.columns=['nb_products','last_order_id','sum_pos']

In [8]:
userXproduct.columns = ['nb_products', 'last_order_id', 'sum_pos']
# 列类型转换
userXproduct.nb_products = userXproduct.nb_products.astype(np.int)

In [9]:
userXproduct.head()

,nb_products,last_order_id,sum_pos
202279_33120,5,"(8, 104690)",10
202279_28985,5,"(6, 132412)",16
202279_9327,1,"(3, 2)",3
202279_45918,5,"(7, 2382766)",24
202279_30035,3,"(7, 2382766)",14


In [10]:
userXproduct.last_order_id = userXproduct.last_order_id.map(lambda x: x[1]).astype(np.int)

In [11]:
userXproduct.head()

,nb_products,last_order_id,sum_pos
202279_33120,5,104690,10
202279_28985,5,132412,16
202279_9327,1,2,3
202279_45918,5,2382766,24
202279_30035,3,2382766,14


In [12]:
userXproduct.sum_pos = userXproduct.sum_pos.astype(np.int)
print('user X product feat', len(userXproduct))
del priors

user X product feat 13307953


In [16]:
orders_train.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,train,11,4,8,14.0
25,1492625,2,train,15,1,11,30.0
49,2196797,5,train,5,0,11,6.0
74,525192,7,train,21,2,11,6.0
78,880375,8,train,4,1,14,10.0


In [17]:
# 从orders划分训练集（用户近期的购买数据）和测试集（用户最后一天的购买数据）
orders_train = orders[orders['eval_set'] == 'train']
# orders_test=orders[orders['eval_set']=='test']
# train数据以(order_id,product_id)为key inplace=True在原数据上修改 drop=False不删除原列
train.set_index(['order_id', 'product_id'], inplace=True, drop=False)

In [21]:
def feat_deal(selected_orders, labels_given=False):
    print('build candidate list')
    order_list = []
    product_list = []
    labels = []
    i = 0
    for row in selected_orders.itertuples():
        i += 1
        if i % 10000 == 0:
            print('dealed rows:', i)
        order_id = row.order_id
        user_id = row.user_id
        # user_id的不重复商品集合
        # user_products=users[users.user_id==user_id].all_products
        user_products = users.all_products[user_id]
        # 产品list，即order_id的候选集
        product_list += user_products
        # 每个product对应当前的order_id,即pair(product_id,order_id)
        order_list += [order_id] * len(user_products)
        # 指定label 如果用户商品在train中那么为1
        if labels_given:
            labels += [(order_id, pid) in orders_train.index for pid in user_products]

    df = pd.DataFrame({'order_id': order_list, 'product_id': product_list}, dtype=np.str)
    labels = np.array(labels, dtype=np.int)
    del order_list
    del product_list

    # 获取user相关特征
    print("user related feat")
    df['user_id'] = df['order_id'].map(orders.user_id)
    # 用户总订单数量
    df['user_total_orders']=df['user_id'].map(users.nb_orders)
    # 用户购买的总商品数
    df['user_total_items']=df['user_id'].map(users.total_items)
    # 用户购买的去重的总商品数
    df['total_distinct_items']=df['user_id'].map(users.total_distinct_items)
    df['user_average_days_between_orders']=df['user_id'].map(users.average_days_between_orders)
    df['user_average_basket']=df['user_id'].map(users.average_basket)

    # 获取订单相关特征
    print('order related feat')
    df['order_hour_of_day']=df['order_id'].map(orders.order_hour_of_day)
    df['days_since_prior_order'] = df.order_id.map(orders.days_since_prior_order)
    df['days_since_ratio'] = df.days_since_prior_order / df.user_average_days_between_orders

    # 商品相关特征
    print('product related feat')
    df['aisle_id'] = df.product_id.map(products.aisle_id)
    df['department_id'] = df.product_id.map(products.department_id)
    df['product_orders'] = df.product_id.map(products.orders).astype(np.int32)
    df['product_reorders'] = df.product_id.map(products.reorders)
    df['product_reorder_rate'] = df.product_id.map(products.reorder_rate)

    # 用户和商品的
    print('user_X_product related features')
    # 组合user_id product_id
    df['z']=df.user_id+'_'+df.product_id
    # 删除user_id
    df.drop(['user_id'],inplace=True,axis=1)
    df['UP_orders'] = df.z.map(userXproduct.nb_products)
    df['UP_orders_ratio'] = (df.UP_orders / df.user_total_orders).astype(np.float32)
    # 共同最后一个订单
    df['UP_last_order_id'] = df.z.map(userXproduct.last_order_id)
    # 物品在该用户订单中的平均位置
    df['UP_average_pos_in_cart'] = (df.z.map(userXproduct.sum_pos) / df.UP_orders).astype(np.float32)
    # 最后一次购买这个物品在倒数第几个订单  [1,1,1,0,1,0,1,0,0]
    df['UP_orders_since_last'] = df.user_total_orders - df.UP_last_order_id.map(orders.order_number)
    # 当前订单与最后订单时间差异（hour）
    df['UP_delta_hour_vs_last'] = abs(df.order_hour_of_day - df.UP_last_order_id.map(orders.order_hour_of_day)).map(
        lambda x: min(x, 24 - x)).astype(np.int8)
    df.drop(['UP_last_order_id', 'z'], axis=1, inplace=True)
    print(df.dtypes)
    print(df.memory_usage())
    return df, labels

f_to_use = ['user_total_orders', 'user_total_items', 'total_distinct_items',
            'user_average_days_between_orders', 'user_average_basket',
            'order_hour_of_day', 'days_since_prior_order', 'days_since_ratio',
            'aisle_id', 'department_id', 'product_orders', 'product_reorders',
            'product_reorder_rate', 'UP_orders', 'UP_orders_ratio',
            'UP_average_pos_in_cart', 'UP_orders_since_last',
            'UP_delta_hour_vs_last']

In [ ]:
df_train,labels=feat_deal(orders_train,True)
print('Train_columns',df_train.columns)

build candidate list
dealed rows: 10000
dealed rows: 20000
dealed rows: 30000
dealed rows: 40000
dealed rows: 50000
dealed rows: 60000
dealed rows: 70000
dealed rows: 80000
dealed rows: 90000
dealed rows: 100000
dealed rows: 110000
dealed rows: 120000
dealed rows: 130000
user related feat
order related feat
product related feat
user_X_product related features


In [ ]:
# 保存结果 index=False不保存index
df_train.to_csv(out_dir+'train_feat.csv',index=False)
np.save(out_dir+'label.npy',labels)
print('ok!')

In [20]:
users.head()

,total_items,all_products,total_distinct_items,average_days_between_orders,nb_orders,average_basket
user_id,,,,,,
1,59,"{38928, 49235, 14084, 46149, 12427, 26088, 359...",18,19.000000,11,5.363636
10,143,"{25931, 22035, 47380, 35973, 44359, 21174, 149...",94,21.799999,6,23.833333
100,27,"{6975, 44278, 24852, 38689, 24221, 48628, 3854...",20,26.799999,6,4.500000
1000,103,"{18889, 35108, 44632, 47734, 38374, 47439, 663...",32,13.428572,8,12.875000
10000,1092,"{24489, 11520, 49235, 48974, 22035, 15261, 318...",256,4.527778,73,14.958904
